In [1]:
import numpy as np
import matplotlib.mlab as mlab
# import matplotlib.pyplot as plt
# %matplotlib inline

In [2]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext()

In [3]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [22]:
# Full original training set
raw_full_train = sc.textFile("dac/train.txt")

# Final test set split
raw_final_test = sc.textFile("dac/split/test.txt")  # Do not touch during training

# Training set splits
raw_test_3m = sc.textFile("dac/split/test_3m.txt")
raw_train_5m = sc.textFile("dac/split/train_5m.txt")
raw_validation_2m = sc.textFile("dac/split/train_5m.txt")

# Debug set
raw_small_train = sc.textFile("dac/small-train.txt")

In [23]:
def convert_value(index, value):
    if index < 14:
        return int(value) if value else None
    else:
        return value if value else None

def convert_line(line):
    return [convert_value(i, value) for i, value in enumerate(line.split("\t"))]

In [24]:
# Change the data types of the datasets so that the RDD's include Int's and Strings. 
full_train = raw_full_train.map(convert_line)
final_test = raw_final_test.map(convert_line)
test_3m = raw_test_3m.map(convert_line)
train_5m = raw_train_5m.map(convert_line)
validation_2m = raw_validation_2m.map(convert_line)

debug = raw_small_train.map(convert_line)

In [25]:
debug.map(lambda row: row[0]).histogram([0,1,2])
# data.map(lambda row: row[0]).histogram([0, 1, 2])
# data.map(lambda row: row[2]).histogram(10)
# a = data.map(lambda row: row[2]).histogram([-10,0,5,10,50,100,200,500,1000,2000,10000,50000])

([0, 1, 2], [77337, 22663])

In [31]:
def show_histogram(col):
    bins, counts = col.histogram(10)
    total = sum(counts)    
    print "bins=%s, counts=%s (total=%d)" % ( bins, counts, total)
    # TODO: display graph of histogram
    # TODO: better buckets for histogram (smart sub-dividing)
        #sum the counts
        #max of the counts
        #if  > 25%

def bin_range_labels(bins):
    return ["%s-%s" % (bins[i], bins[i + 1]) for i in range(len(bins) - 1)]

In [27]:
def get_column_num(data, col_num):
    return data.map(lambda row: row[col_num])

def column_filter_null(column):
    return column.filter(lambda row: row is not None)

def column_integer_iter(data):
    for i in range(1,14):
         yield get_column_num(data, i)

In [32]:
def all_columns_histograms(data):
    for col in column_integer_iter(debug):
        show_histogram(col)

def print_column_summary(col_num, mean, stdev):
    print("Column #%d: Mean=%0.3f, Stdev=%0.3f" % (col_num, mean, stdev))
        
def int_columns_summary_stats(data):
    for i, col in enumerate(column_integer_iter(debug)):
        # TODO: add skew, kurtosis
        mean = column_filter_null(col).mean()
        stdev = column_filter_null(col).sampleStdev()
        print_column_summary(i + 2, mean, stdev)


In [33]:
all_columns_histograms(debug)

bins=[0.0, 55.6, 111.2, 166.8, 222.4, 278.0, 333.6, 389.2, 444.8, 500.40000000000003, 556], counts=[55236, 295, 34, 10, 5, 3, 2, 0, 1, 1] (total=55587)
bins=[-2.0, 1850.4, 3702.8, 5555.200000000001, 7407.6, 9260.0, 11112.400000000001, 12964.800000000001, 14817.2, 16669.600000000002, 18522], counts=[98530, 1397, 52, 12, 7, 0, 1, 0, 0, 1] (total=100000)
bins=[0.0, 6553.5, 13107.0, 19660.5, 26214.0, 32767.5, 39321.0, 45874.5, 52428.0, 58981.5, 65535], counts=[80863, 11, 1, 20, 0, 0, 0, 0, 1, 2] (total=80898)
bins=[0.0, 41.7, 83.4, 125.10000000000001, 166.8, 208.5, 250.20000000000002, 291.90000000000003, 333.6, 375.3, 417], counts=[79210, 1178, 44, 8, 12, 6, 5, 0, 0, 3] (total=80466)
bins=[0.0, 174112.8, 348225.6, 522338.39999999997, 696451.2, 870564.0, 1044676.7999999999, 1218789.5999999999, 1392902.4, 1567015.2, 1741128], counts=[93216, 1145, 491, 218, 88, 44, 19, 11, 5, 3] (total=95240)
bins=[0, 1629, 3258, 4887, 6516, 8145, 9774, 11403, 13032, 14661, 16290], counts=[74200, 536, 92, 36,

In [34]:
int_columns_summary_stats(debug)

Column #2: Mean=3.769, Stdev=10.451
Column #3: Mean=112.864, Stdev=401.523
Column #4: Mean=40.745, Stdev=538.819
Column #5: Mean=8.280, Stdev=10.836
Column #6: Mean=17592.599, Stdev=65797.898


KeyboardInterrupt: 